# OpenAI – Unified Endpoint Examples

## Non-Streaming with Function Calling

In [148]:
import os
from openai import OpenAI
from javelin_sdk import JavelinClient, JavelinConfig
from dotenv import load_dotenv

load_dotenv()

def init_openai_client():
    api_key = os.getenv("OPENAI_API_KEY")
    return OpenAI(api_key=api_key)

def init_javelin_client(openai_client, route_name="openai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_openai(openai_client, route_name=route_name)
    return client

def openai_function_call_non_stream():
    client = init_openai_client()
    # Register with the unified endpoint
    init_javelin_client(client)
    response = client.chat.completions.create(
        model="o3-mini",  # Latest o1 model
        messages=[
            {"role": "user", "content": "What is the current weather in New York?"}
        ],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Retrieves current weather information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City and state (e.g., New York, NY)"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"]
                        }
                    },
                    "required": ["location"]
                }
            }
        ],
        function_call="auto"
    )
    print("OpenAI Non-Streaming Response:")
    print(response.model_dump_json(indent=2))

if __name__ == "__main__":
    openai_function_call_non_stream()


OpenAI Non-Streaming Response:
{
  "id": "chatcmpl-B5BpQmrAdVu4uOUUEniPwLvD9omRu",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": {
          "arguments": "{\"location\": \"New York, NY\", \"unit\": \"fahrenheit\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1740576808,
  "model": "o3-mini-2025-01-31",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_42bfad963b",
  "usage": {
    "completion_tokens": 98,
    "prompt_tokens": 76,
    "total_tokens": 174,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 64,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
   

## Streaming with Function Calling

In [12]:
import os
from openai import OpenAI
from javelin_sdk import JavelinClient, JavelinConfig

def init_openai_client():
    api_key = os.getenv("OPENAI_API_KEY")
    return OpenAI(api_key=api_key)

def init_javelin_client(openai_client, route_name="openai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_openai(openai_client, route_name=route_name)
    return client

def openai_function_call_stream():
    client = init_openai_client()
    init_javelin_client(client)
    stream = client.chat.completions.create(
        model="o1",
        messages=[
            {"role": "user", "content": "Tell me a fun fact and then call a function."}
        ],
        functions=[
            {
                "name": "tell_fun_fact",
                "description": "Returns a fun fact",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "fact": {
                            "type": "string",
                            "description": "A fun fact about the topic"
                        }
                    },
                    "required": ["fact"]
                }
            }
        ],
        function_call="auto",
        stream=True
    )
    collected = []
    print("stream>>>",stream)
    for chunk in stream:
        # Each chunk may contain a delta with part of the response
        delta = chunk.choices[0].delta
        print(chunk)
        if hasattr(delta, "content") and delta.content:
            collected.append(delta.content)
    print("OpenAI Streaming Response:")
    print("".join(collected))

if __name__ == "__main__":
    openai_function_call_stream()


stream>>> <openai.Stream object at 0x10b0e78f0>
ChatCompletionChunk(id='chatcmpl-B54xyXCjyKyitk5wxg3DDQ7t6YA8a', choices=[Choice(delta=ChoiceDelta(content=None, function_call=ChoiceDeltaFunctionCall(arguments='', name='tell_fun_fact'), refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1740550430, model='o1-2024-12-17', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_30d1ee942c', usage=None)
ChatCompletionChunk(id='chatcmpl-B54xyXCjyKyitk5wxg3DDQ7t6YA8a', choices=[Choice(delta=ChoiceDelta(content=None, function_call=ChoiceDeltaFunctionCall(arguments='{"', name=None), refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1740550430, model='o1-2024-12-17', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_30d1ee942c', usage=None)
ChatCompletionChunk(id='chatcmpl-B54xyXCjyKyitk5wxg3DDQ7t6YA8a', choices=[Choice(delta=ChoiceDelta(content

## Structured output calling Example

In [162]:
import os
import json
from openai import OpenAI
from javelin_sdk import JavelinClient, JavelinConfig
from dotenv import load_dotenv

load_dotenv()

def init_openai_client():
    api_key = os.getenv("OPENAI_API_KEY")
    return OpenAI(api_key=api_key)

def init_javelin_client(openai_client, route_name="openai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_openai(openai_client, route_name=route_name)
    return client

def openai_structured_output_call_generic():
    # Initialize clients
    openai_client = init_openai_client()
    init_javelin_client(openai_client)
    
    # Create messages with a system instruction to output valid JSON
    messages = [
        {
            "role": "system",
            "content": "You are an assistant that always responds in valid JSON format without any additional text."
        },
        {
            "role": "user",
            "content": (
                "Provide a generic example of structured data output in JSON format. "
                "The JSON should include the keys: 'id', 'name', 'description', "
                "and 'attributes' (which should be a nested object with arbitrary key-value pairs)."
            )
        }
    ]
    
    response = openai_client.chat.completions.create(
        model="o3-mini", # can use o1 model as well
        messages=messages,
    )
    
    # Print the full API response for reference
    print("Structured Output (JSON) Response:")
    print(response.model_dump_json(indent=2))
    
    # Extract the reply content using attribute access
    try:
        reply_content = response.choices[0].message.content
    except (IndexError, AttributeError) as e:
        print("Error extracting message content:", e)
        reply_content = ""
    
    # Attempt to parse the reply content as JSON
    try:
        json_output = json.loads(reply_content)
        print("\nParsed JSON Output:")
        print(json.dumps(json_output, indent=2))
    except Exception as e:
        print("\nFailed to parse JSON output. Error:", e)
        print("Raw content:", reply_content)

if __name__ == "__main__":
    openai_structured_output_call_generic()


Structured Output (JSON) Response:
{
  "id": "chatcmpl-B5D3I4NnbFAscgmT00ViE1kuocr8j",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\n  \"id\": \"1234\",\n  \"name\": \"Sample Item\",\n  \"description\": \"This item serves as a generic example of structured data output in JSON format.\",\n  \"attributes\": {\n    \"color\": \"blue\",\n    \"size\": \"medium\",\n    \"weight\": 2.5,\n    \"tags\": [\"example\", \"generic\", \"json\"]\n  }\n}",
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1740581512,
  "model": "o3-mini-2025-01-31",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_42bfad963b",
  "usage": {
    "completion_tokens": 348,
    "prompt_tokens": 71,
    "total_tokens": 419,
    "completion_tokens_details": {
      "accepted_predict

#  Azure OpenAI – Unified Endpoint Examples

## Non-Streaming with Function Calling

In [152]:
import os
from openai import AzureOpenAI
from javelin_sdk import JavelinClient, JavelinConfig

def init_azure_client():
    azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
    return AzureOpenAI(
        api_version="2023-07-01-preview",
        azure_endpoint="https://javelinpreview.openai.azure.com",
        api_key=azure_api_key
    )

def init_javelin_client_azure(azure_client, route_name="azureopenai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_azureopenai(azure_client, route_name=route_name)
    return client

def azure_function_call_non_stream():
    azure_client = init_azure_client()
    init_javelin_client_azure(azure_client)
    response = azure_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "Schedule a meeting at 10 AM tomorrow."}
        ],
        functions=[
            {
                "name": "schedule_meeting",
                "description": "Schedules a meeting in the calendar",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "time": {"type": "string", "description": "Meeting time (ISO format)"},
                        "date": {"type": "string", "description": "Meeting date (YYYY-MM-DD)"}
                    },
                    "required": ["time", "date"]
                }
            }
        ],
        function_call="auto"
    )
    print("Azure OpenAI Non-Streaming Response:")
    print(response.to_json())

if __name__ == "__main__":
    azure_function_call_non_stream()


Azure OpenAI Non-Streaming Response:
{
  "id": "chatcmpl-B5BuG5zyeIE3eahuneeL9g5Ph7pXh",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\"date\":\"2023-11-07\",\"time\":\"10:00:00\"}",
          "name": "schedule_meeting"
        }
      },
      "content_filter_results": {}
    }
  ],
  "created": 1740577108,
  "model": "gpt-4o-2024-11-20",
  "object": "chat.completion",
  "system_fingerprint": "fp_b705f0c291",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 73,
    "total_tokens": 101,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  },
  "javelin": {
    "a

## Structured output calling - Azure Open Ai Example

In [164]:
import os
import json
import re
from openai import AzureOpenAI
from javelin_sdk import JavelinClient, JavelinConfig
from dotenv import load_dotenv

load_dotenv()

def init_azure_client():
    azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
    return AzureOpenAI(
        api_version="2023-07-01-preview",
        azure_endpoint="https://javelinpreview.openai.azure.com",
        api_key=azure_api_key
    )

def init_javelin_client_azure(azure_client, route_name="azureopenai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_azureopenai(azure_client, route_name=route_name)
    return client

def extract_json_from_markdown(text: str) -> str:
    """
    Extracts JSON content from a markdown code block if present.
    For example, removes leading and trailing triple backticks.
    """
    match = re.search(r"```(?:json)?\s*(\{.*\})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    return text.strip()

def azure_structured_output_call():
    azure_client = init_azure_client()
    init_javelin_client_azure(azure_client)

    # System message enforces structured JSON output without extra text
    messages = [
        {
            "role": "system",
            "content": "You are an assistant that always responds in valid JSON format without any additional text."
        },
        {
            "role": "user",
            "content": (
                "Provide structured data in JSON format. "
                "The JSON should contain the following keys: 'id' (integer), 'title' (string), "
                "'description' (string), and 'metadata' (a nested object with arbitrary key-value pairs)."
            )
        }
    ]

    response = azure_client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    # Print the full API response for reference
    print("Structured Output (JSON) Response:")
    print(response.to_json())

    # Extract and clean the reply content
    try:
        reply_content = response.choices[0].message.content  # Get the raw response text
        # Remove markdown code fences if present
        reply_content_clean = extract_json_from_markdown(reply_content)
        json_output = json.loads(reply_content_clean)  # Parse as JSON
        print("\nParsed JSON Output:")
        print(json.dumps(json_output, indent=2))
    except Exception as e:
        print("\nFailed to parse JSON output. Error:", e)
        print("Raw content:", reply_content)

if __name__ == "__main__":
    azure_structured_output_call()


Structured Output (JSON) Response:
{
  "id": "chatcmpl-B5DB37rD9OIR5BnSbP5iaFonLe2I8",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "```json\n{\n  \"id\": 1,\n  \"title\": \"Sample Data\",\n  \"description\": \"This is an example of structured JSON data.\",\n  \"metadata\": {\n    \"author\": \"John Doe\",\n    \"created_at\": \"2023-10-01\",\n    \"tags\": [\"example\", \"json\", \"data\"],\n    \"version\": 1.0\n  }\n}\n```",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "detected": false,
          "filtered": false
        },
        "protected_material_text": {
          "detected": false,
          "filtered": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
       

## Streaming with Function Calling

In [28]:
import os
from openai import AzureOpenAI
from javelin_sdk import JavelinClient, JavelinConfig

def init_azure_client():
    azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
    return AzureOpenAI(
        api_version="2023-07-01-preview",
        azure_endpoint="https://javelinpreview.openai.azure.com",
        api_key=azure_api_key
    )

def init_javelin_client_azure(azure_client, route_name="azureopenai_univ"):
    javelin_api_key = os.getenv("JAVELIN_API_KEY")
    config = JavelinConfig(javelin_api_key=javelin_api_key)
    client = JavelinClient(config)
    client.register_azureopenai(azure_client, route_name=route_name)
    return client

def azure_function_call_non_stream():
    azure_client = init_azure_client()
    init_javelin_client_azure(azure_client)
    stream = azure_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "Schedule a meeting at 10 AM tomorrow."}
        ],
        functions=[
            {
                "name": "schedule_meeting",
                "description": "Schedules a meeting in the calendar",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "time": {"type": "string", "description": "Meeting time (ISO format)"},
                        "date": {"type": "string", "description": "Meeting date (YYYY-MM-DD)"}
                    },
                    "required": ["time", "date"]
                }
            }
        ],
        function_call="auto",
        stream=True
    )
    print("Azure OpenAI Non-Streaming Response:")
    for chunk in stream:
        print(chunk)

if __name__ == "__main__":
    azure_function_call_non_stream()


Azure OpenAI Non-Streaming Response:
ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
ChatCompletionChunk(id='chatcmpl-B557pUz4IkUX7UuDCRvgKypQetQ6H', choices=[Choice(delta=ChoiceDelta(content=None, function_call=ChoiceDeltaFunctionCall(arguments='', name='schedule_meeting'), refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1740551041, model='gpt-4o-2024-11-20', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_b705f0c291', usage=None)
ChatCompletionChunk(id='chatcmpl-B557pUz4Ik

# OpenAI – Regular Route Endpoint Examples

## Non-Streaming with Function Calling

In [170]:
import os
from javelin_sdk import JavelinClient, JavelinConfig, RouteNotFoundError

# Retrieve API keys from environment variables
javelin_api_key = os.getenv('JAVELIN_API_KEY')
llm_api_key = os.getenv("OPENAI_API_KEY")

if not javelin_api_key or not llm_api_key:
    raise ValueError("Both JAVELIN_API_KEY and OPENAI_API_KEY must be set.")

print("OpenAI LLM API Key:", llm_api_key)

# Configure the Javelin client
config = JavelinConfig(
    base_url="https://api-dev.javelin.live",
    javelin_api_key=javelin_api_key,
    llm_api_key=llm_api_key,
)
client = JavelinClient(config)
print("Successfully connected to Javelin Client for OpenAI")

# Prepare query data with function calling support
query_data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant that translates English to French."},
        {"role": "user", "content": "AI has the power to transform humanity and make the world a better place."},
    ],
    "functions": [
        {
            "name": "translate_text",
            "description": "Translates English text to French",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "Text to translate"
                    }
                },
                "required": ["text"]
            }
        }
    ],
    "function_call": "auto"
}

# Query the LLM using the "openai" route
try:
    response = client.query_route("openai", query_data)
    print("Response from OpenAI Regular Endpoint:")
    print(response)
except RouteNotFoundError:
    print("Route 'openai' Not Found")
except Exception as e:
    print("Error querying OpenAI endpoint:", e)


## Streaming with Function Calling

In [171]:
import os
from javelin_sdk import JavelinClient, JavelinConfig, RouteNotFoundError

# Retrieve API keys from environment variables
javelin_api_key = os.getenv('JAVELIN_API_KEY')
llm_api_key = os.getenv("OPENAI_API_KEY")

if not javelin_api_key or not llm_api_key:
    raise ValueError("Both JAVELIN_API_KEY and OPENAI_API_KEY must be set.")

print("OpenAI LLM API Key:", llm_api_key)

# Configure the Javelin client
config = JavelinConfig(
    base_url="https://api-dev.javelin.live",
    javelin_api_key=javelin_api_key,
    llm_api_key=llm_api_key,
)
client = JavelinClient(config)
print("Successfully connected to Javelin Client for OpenAI")

# Prepare query data with function calling support
query_data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant that translates English to French."},
        {"role": "user", "content": "AI has the power to transform humanity and make the world a better place."},
    ],
    "functions": [
        {
            "name": "translate_text",
            "description": "Translates English text to French",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "Text to translate"
                    }
                },
                "required": ["text"]
            }
        }
    ],
    "function_call": "auto",
    # "stream": True
}

# Query the LLM using the "openai" route
try:
    response = client.query_route("openai", query_data)
    print("Response from OpenAI Regular Endpoint:")

    # If streaming is enabled, iterate over the stream of response chunks
    if query_data.get("stream"):
        for chunk in response:
            print(chunk)
    else:
        print(response)
except RouteNotFoundError:
    print("Route 'openai' Not Found")
except Exception as e:
    print("Error querying OpenAI endpoint:", e)
